In [6]:
import os
# from PyARMViz.PyARMViz import adjacency_parallel_coordinate_plot
# from PyARMViz.PyARMViz import metadata_scatter_plot
import arulesviz
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

#### set defaults
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)


print(os.getcwd())
os.chdir('/home/ci95poh/PycharmProjects/DataMining/BI/AssociateRules')
# df = pd.read_csv('Groceries_dataset_R.csv')

# df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
# df = pd.read_csv('Online_Retail.csv')
df = pd.read_excel('Online_Retail.xlsx')
print('data loaded')

/home/ci95poh/PycharmProjects/DataMining/BI/AssociateRules
data loaded


In [7]:


#### data preparation ####
# remove blanks
df['Description'] = df['Description'].str.strip()
# drop rows (axis=0) containing no values
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
# make sure, ivoice number is of type string
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
# remove all rows, wheer invoice number contains an 'C'
df = df[~df['InvoiceNo'].str.contains('C')]

# [df['Country'] =="France"]		"only" records from France,
# .groupby							group first by InvoiceNo, then Description
# ['Quantity'].sum()				sum it up in new columns quantity
# .unstack()						make one line for each InvoiceNo
# reset_index()
# fillna(0)							set all cells with no value to 0
# .set_index						male new index
basket = (df[df['Country'] == "France"]
		  .groupby(['InvoiceNo', 'Description'])['Quantity']
		  .sum().unstack().reset_index().fillna(0)
		  .set_index('InvoiceNo'))


# make one hot encoding
def encode_units(x):
	if x <= 0:
		return 0
	if x >= 1:
		return 1


#applymap							a function that accepts and returns a scalar to every element of a DataFrame.
basket_sets = basket.applymap(encode_units)
# drop column 'POSTAGE'
basket_sets.drop('POSTAGE', inplace=True, axis=1)

print('data preparation completed')

data preparation completed


In [ ]:
# model
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

# sample rules
subrules_1 = rules[(rules['lift'] >= 6) & (rules['confidence'] >= 0.8)]
subrules_1.head()

g = arulesviz.Arulesviz(basket_sets, 0.001, 0.3, 12, products_to_drop=[])
g.create_rules()
g.plot_graph(width=1800, directed=False, charge=-150, link_distance=20)

In [ ]:
##### plot rules ####
arulesviz.arulesviz.GridHeatMap(g)
# metadata_scatter_plot(subrules_1)
# adjacency_parallel_coordinate_plot(subrules_1)